In [1]:
!pip install -q datasets
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 1. 한국어 데이터셋

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
data_dir = '/content/drive/MyDrive/공ㅂ/data/'

In [71]:
import glob

file_names = glob.glob(data_dir+'*/*.json')
len(file_names)

6375

In [72]:
import pandas as pd
import json
from tqdm import tqdm

df = pd.DataFrame(columns = ['instructions','output'])

for file_name in tqdm(file_names):
  # 1. load json
  with open(file_name, 'r') as f:
    json_data = json.load(f)

  # 2. make instruction-output form
  for i, dic in enumerate(json_data['document'][0]['utterance']):
    # print(dic['original_form'])
    if i==0:
      last_sentence = dic['original_form']
    else:
      if last_sentence != dic['original_form']:
        df.loc[i] = [last_sentence, dic['original_form']]
        last_sentence = dic['original_form']

100%|██████████| 6375/6375 [20:41<00:00,  5.13it/s]


In [73]:
df

,instructions,output
1,예.,그~ 선생님은
2,그~ 선생님은,혹시 이제 결혼하셨는데
3,혹시 이제 결혼하셨는데,어~ 혹시 뭐 이상형이 어떻게 되셨나요?
4,어~ 혹시 뭐 이상형이 어떻게 되셨나요?,네. 저는
5,네. 저는,어~
...,...,...
760,알겠어 그럼 그렇게 할게.,응.
761,응.,(())
762,(()),알겠어.
684,(()),낙이 없어 보여.


In [76]:
other_data_dir = '/content/drive/MyDrive/공ㅂ/일상대화'

file_names = glob.glob(other_data_dir+'/*.json')
len(file_names)

3

In [77]:
with open(file_names[1], 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [78]:
def process_file(file_name):
    rows = []
    try:
        with open(file_name, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        for dic in json_data:
          if dic['user_utterance'] == "null":
              rows.append({'instructions': None, 'output': dic['system_utterance']})
          else:
              rows.append({'instructions': dic['user_utterance'], 'output': dic['system_utterance']})

    except:
        print(file_name)

    return rows

all_rows = []
for file_name in file_names:
    file_rows = process_file(file_name)
    all_rows.extend(file_rows)

/content/drive/MyDrive/공ㅂ/일상대화/output_daily_3rd.json


In [80]:
new_df = pd.DataFrame(all_rows, columns=['instructions', 'output'])

combined_df = pd.concat([df, new_df], ignore_index=True)

In [81]:
combined_df.to_json('/content/drive/MyDrive/공ㅂ/train_data.json',orient='records')

In [82]:
combined_df

,instructions,output
0,예.,그~ 선생님은
1,그~ 선생님은,혹시 이제 결혼하셨는데
2,혹시 이제 결혼하셨는데,어~ 혹시 뭐 이상형이 어떻게 되셨나요?
3,어~ 혹시 뭐 이상형이 어떻게 되셨나요?,네. 저는
4,네. 저는,어~
...,...,...
13909,뭘 먹을까?,기승전 고기 아니겠습니까? 무한리필 고기집 추천합니다.
13910,"근데, 바로 근무해야 하는데 좀 부담되네.",그럼 몸보신을 위해서 추어탕 어떻습니까?
13911,"그래, 나름 괜찮네.",그럼요. 괜찮은 선택을 했다고 생각하실 거예요.
13912,오늘은 날이 좋네.,점심은 밖에서 맛있는거 사드세요!


In [18]:
import datasets

data = datasets.load_dataset('json', data_files='/content/drive/MyDrive/공ㅂ/train_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
data

DatasetDict({
    train: Dataset({
        features: ['instructions', 'output'],
        num_rows: 761
    })
})

In [83]:
data = data.map(
    lambda x: {'text': f"{ x['instructions']}\n\n 정답: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

# 2. Tokenizer

In [84]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [85]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [86]:
data['train'][0]['text']

'아니 그 내가 그때 애기했잖아.\n\n 정답: 그<|endoftext|>'

In [88]:
from peft import get_peft_model,IA3Config

config=IA3Config(task_type="CAUSAL_LM")#(target_modules=["query_key_value"],

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1433600 || all params: 6603581440 || trainable%: 0.02170943166258581


In [89]:
# GPU 할당이 어떻게 되어있는지 체킹하여 봅니다.
for i in model.named_parameters():
    print(f"{i[0]} -> {i[1].device}")

base_model.model.gpt_neox.embed_in.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.base_layer.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.base_layer.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.ia3_l.default -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.base_layer.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        num_train_epochs=4,
        max_steps=15930, # 가장 최적화된 15930 step까지 훈련되었습니다.
        learning_rate=5e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim='adamw_torch'),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,6.453500
20,6.322400
30,6.942900
40,6.489000
50,6.553200
60,6.469800
70,5.994000
80,6.455700
90,6.887700
100,6.473100
